In [1]:
import os
import shutil
import pandas as pd
import sqlite3

import utils

from database import db, mongodb

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dbname = 'curontab.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()
c.execute('PRAGMA foreign_keys=true')

In [3]:
client = pymongo.MongoClient('localhost',27017)
db = client.curontab
co = db.dataset

In [4]:
try:
    c.execute('drop table model')
    c.execute('drop table upload_data')
except:
    pass

q = '''
create table model (
    id integer primary key,
    name varcher(64),
    validation_table varcher(64),
    dataset_id varcher(32),
    dataset_name varcher(64)
)
'''
c.execute(q)

q = '''
create table upload_data (
    id integer primary key,
    name varcher(64),
    path varcher(64)
)
'''
c.execute(q)
conn.commit()

co.remove({})

/home/fytroo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


{'n': 0, 'ok': 1}

In [5]:
dataset_name = 'eyes'
dataset_dir = os.path.join('dataset',dataset_name)
labels = list(set(os.listdir(dataset_dir)))
labels.sort()
id2label = {i:label for i,label in enumerate(labels) }
label2id = {label:i for i,label in enumerate(labels) }

In [6]:
dataset_info = {
    'labels': list(labels),
    'label2id': label2id,
    'name': dataset_name,
    'data':[],
    'models':[],
}
dataset_info

{'data': [],
 'label2id': {'AB': 0, 'N': 1},
 'labels': ['AB', 'N'],
 'models': [],
 'name': 'eyes'}

In [7]:
dataset_id = co.insert(dataset_info)
dataset_id

/home/fytroo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5a26058ab037cbae7a7b2739')

In [8]:
data_table = 'data_{}'.format(dataset_id)
data_df = utils.df_fromdir_classed(dataset_dir)
data_df['path'] = utils.format_dirname(dataset_dir) + data_df['label'] + '/' + data_df['fname']
data_df.to_sql(data_table, conn)
co.update({'_id':dataset_id},{'$set':{'data':data_table}})

/home/fytroo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """


{'n': 1, 'nModified': 1, 'ok': 1, 'updatedExisting': True}

In [9]:
dataset_name = 'brain'
dataset_dir = os.path.join('dataset',dataset_name)
labels = set(os.listdir(dataset_dir))
id2label = {i:label for i,label in enumerate(labels) }
label2id = {label:i for i,label in enumerate(labels) }

dataset_info = {
    'labels': list(labels),
    'label2id': label2id,
    'name': dataset_name,
    'data':[],
    'models':[],
}
dataset_id = co.insert(dataset_info)
data_table = 'data_{}'.format(dataset_id)
data_df = utils.df_fromdir_classed(dataset_dir)
data_df['path'] = utils.format_dirname(dataset_dir) + data_df['label'] + '/' + data_df['fname']
data_df.to_sql(data_table, conn)
co.update({'_id':dataset_id},{'$set':{'data':data_table}})

/home/fytroo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  
/home/fytroo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


{'n': 1, 'nModified': 1, 'ok': 1, 'updatedExisting': True}

In [10]:
conn.commit()
conn.close()